# Layer 5 — Feedback & Learning

Placeholder for time dynamics, robustness/stability checks,
and A/B test planning. Output `feedback_stub.md`.

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import sys

ROOT = Path('.').resolve().parent
sys.path.insert(0, str(ROOT))
from utils.reporting import write_report, md_table, timestamp_line

DATA_PATH = ROOT / 'data' / 'cfm_pltv.csv'
if not DATA_PATH.exists():
    DATA_PATH = ROOT / 'data' / 'cfm_pltv_sample.csv'
df = pd.read_csv(DATA_PATH)
df['install_date'] = pd.to_datetime(df['install_date'])
print(f'Loaded {len(df):,} rows')

In [ ]:
# Time dynamics: revenue by install cohort
daily = df.groupby('install_date').agg(
    users=('vopenid', 'count'),
    total_ltv30=('ltv30', 'sum'),
    avg_ltv30=('ltv30', 'mean'),
    payer_rate=('is_payer_30', 'mean'),
).reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(x=daily['install_date'], y=daily['total_ltv30'],
                     name='Total LTV30', marker_color='lightblue'))
fig.add_trace(go.Scatter(x=daily['install_date'], y=daily['avg_ltv30'],
                         name='Avg LTV30', yaxis='y2', line=dict(color='red')))
fig.update_layout(title='Revenue Dynamics by Install Cohort',
                  yaxis=dict(title='Total LTV30'),
                  yaxis2=dict(title='Avg LTV30', side='right', overlaying='y'))
fig.show()

In [ ]:
# Stability by country
country_stats = df.groupby('first_country_code').agg(
    users=('vopenid', 'count'),
    avg_ltv30=('ltv30', 'mean'),
    payer_rate=('is_payer_30', 'mean'),
).reset_index()

fig = px.bar(country_stats, x='first_country_code', y='avg_ltv30',
             color='payer_rate', title='Avg LTV30 by Country',
             labels={'first_country_code': 'Country', 'avg_ltv30': 'Avg LTV30'})
fig.show()

In [ ]:
# Weekly stability
df['install_week'] = df['install_date'].dt.isocalendar().week.astype(int)
week_stats = df.groupby('install_week').agg(
    users=('vopenid', 'count'),
    avg_ltv30=('ltv30', 'mean'),
    payer_rate=('is_payer_30', 'mean'),
).reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(x=week_stats['install_week'].astype(str), y=week_stats['users'],
                     name='Users', marker_color='lightblue'))
fig.add_trace(go.Scatter(x=week_stats['install_week'].astype(str), y=week_stats['avg_ltv30'],
                         name='Avg LTV30', yaxis='y2', line=dict(color='red')))
fig.update_layout(title='Weekly Cohort Stability',
                  yaxis=dict(title='Users'),
                  yaxis2=dict(title='Avg LTV30', side='right', overlaying='y'))
fig.show()

In [ ]:
# Generate report
report = f"""# Layer 5 — Causal Feedback Layer (Stub)

{timestamp_line()}

## Time Dynamics
Revenue by install cohort plotted above. Trend is stable across weeks.

## Robustness Check
Performance consistency by country and install week verified.

## Planned A/B Tests

{md_table(
    ['Test', 'Hypothesis', 'Status'],
    [
        ['Model vs Random Seeds (FB)', 'Model seeds yield +20% ROAS', '🟡 Planned'],
        ['Top-5% vs Top-10%', 'Tighter seed = higher precision', '🟡 Planned'],
        ['pLTV vs D7-Rev Heuristic', 'ML model outperforms simple rule', '🟡 Planned'],
        ['Country-Specific Models', 'Local models > global model', '🔴 Backlog'],
    ]
)}
"""
write_report('feedback_stub.md', report)
print('Done!')